# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [5]:
import pandas as pd

# Carregar as bases de dados
application_data = pd.read_csv('/home/adrianoarchlinux/Downloads/archive/application_record.csv')
credit_data = pd.read_csv('/home/adrianoarchlinux/Downloads/archive/credit_record.csv')

In [7]:
import pandas as pd

# Carregar as bases de dados
application_data = pd.read_csv('/home/adrianoarchlinux/Downloads/archive/application_record.csv')
credit_data = pd.read_csv('/home/adrianoarchlinux/Downloads/archive/credit_record.csv')

# Exibir as primeiras linhas de cada base para uma inspeção inicial
application_data_head = application_data.head()
credit_data_head = credit_data.head()

# Avaliar as variáveis e tipos de dados
application_data_dtypes = application_data.dtypes
credit_data_dtypes = credit_data.dtypes

# Quantificar os valores ausentes
missing_application_data = application_data.isnull().sum()
missing_credit_data = credit_data.isnull().sum()

(application_data_head, credit_data_head, application_data_dtypes, credit_data_dtypes, missing_application_data, missing_credit_data)


(        ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
 0  5008804           M            Y               Y             0   
 1  5008805           M            Y               Y             0   
 2  5008806           M            Y               Y             0   
 3  5008808           F            N               Y             0   
 4  5008809           F            N               Y             0   
 
    AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
 0          427500.0               Working               Higher education   
 1          427500.0               Working               Higher education   
 2          112500.0               Working  Secondary / secondary special   
 3          270000.0  Commercial associate  Secondary / secondary special   
 4          270000.0  Commercial associate  Secondary / secondary special   
 
      NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
 0        Civil marriage   Rented apartm

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [9]:
# Criar listas com os nomes das variáveis e os tipos de dados
variable_names = application_data.columns.tolist()
data_types = application_data.dtypes.tolist()

# Lista para armazenar a quantidade de categorias
num_categories = []

# Iterar pelas variáveis, contar as categorias e armazenar na lista
for column in variable_names:
    num_categories.append(application_data[column].nunique())

# Criar o dataframe de metadados
metadata_df = pd.DataFrame({
    'variable': variable_names,
    'data_type': data_types,
    'qtd_categorias': num_categories
})

metadata_df


,variable,data_type,qtd_categorias
0,ID,int64,438510
1,CODE_GENDER,object,2
2,FLAG_OWN_CAR,object,2
3,FLAG_OWN_REALTY,object,2
4,CNT_CHILDREN,int64,12
5,AMT_INCOME_TOTAL,float64,866
6,NAME_INCOME_TYPE,object,5
7,NAME_EDUCATION_TYPE,object,5
8,NAME_FAMILY_STATUS,object,5
9,NAME_HOUSING_TYPE,object,6


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [11]:
# Identificar variáveis qualitativas que não estão armazenadas como {0, 1} ou {True, False}
qualitative_vars = metadata_df[(metadata_df['data_type'] == 'object') & (metadata_df['qtd_categorias'] > 2)]['variable'].tolist()

# Criar variáveis dummy
application_data_dummies = pd.get_dummies(application_data, columns=qualitative_vars, drop_first=True)

# Eliminar variáveis do tipo str, mantendo apenas suas versões dummy
final_dataframe = application_data_dummies.select_dtypes(include=[int, float, 'uint8'])

final_dataframe.head()


,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS
0,5008804,0,427500.0,-12005,-4542,1,1,0,0,2.0
1,5008805,0,427500.0,-12005,-4542,1,1,0,0,2.0
2,5008806,0,112500.0,-21474,-1134,1,0,0,0,2.0
3,5008808,0,270000.0,-19110,-3051,1,0,1,1,1.0
4,5008809,0,270000.0,-19110,-3051,1,0,1,1,1.0


#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [13]:
# Criar uma coluna 'mau' no application_data com base nos registros de crédito
credit_data['status'] = credit_data['STATUS'].apply(lambda x: 1 if x in ['2', '3', '4', '5'] else 0)
mau_data = credit_data.groupby('ID')['status'].max().reset_index()
mau_data.rename(columns={'status': 'mau'}, inplace=True)

# Mesclar com application_data
application_data = application_data.merge(mau_data, on='ID', how='left')
application_data['mau'] = application_data['mau'].fillna(0).astype(int)


In [17]:
# Listar todas as variáveis no conjunto de dados application_data
print(application_data.columns)


Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'mau'],
      dtype='object')


In [19]:
crosstab_email = pd.crosstab(application_data['FLAG_EMAIL'], application_data['mau'], normalize='index')
crosstab_vehicle = pd.crosstab(application_data['FLAG_OWN_CAR'], application_data['mau'], normalize='index')

print("Tabela Cruzada - FLAG_EMAIL:\n", crosstab_email)
print("\nTabela Cruzada - FLAG_OWN_CAR:\n", crosstab_vehicle)

# Analisar a proporção de mau = 1
email_mau_proportion = crosstab_email[1]
vehicle_mau_proportion = crosstab_vehicle[1]

print("Proporção de Mau = 1 para FLAG_EMAIL:\n", email_mau_proportion)
print("\nProporção de Mau = 1 para FLAG_OWN_CAR:\n", vehicle_mau_proportion)


Tabela Cruzada - FLAG_EMAIL:
 mau                0         1
FLAG_EMAIL                    
0           0.998558  0.001442
1           0.998904  0.001096

Tabela Cruzada - FLAG_OWN_CAR:
 mau                  0         1
FLAG_OWN_CAR                    
N             0.998577  0.001423
Y             0.998627  0.001373
Proporção de Mau = 1 para FLAG_EMAIL:
 FLAG_EMAIL
0    0.001442
1    0.001096
Name: 1, dtype: float64

Proporção de Mau = 1 para FLAG_OWN_CAR:
 FLAG_OWN_CAR
N    0.001423
Y    0.001373
Name: 1, dtype: float64


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [23]:
# Salvar a base de dados processada em um arquivo CSV
final_dataframe.to_csv('./final_dataframe.csv', index=False)
